In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
import ipywidgets as widgets
import sys, os, io, string, shutil, math
from hublib.ui import Submit
from hublib.ui import RunCommand
import hublib.use
%use boost-1.62.0-mpich2-1.3-gnu-4.7.2
%use lammps-31Mar17
import numpy as np
import re
import time
from threading import Thread
from io import StringIO
from ipywidgets import Layout, Box, Label, Output
from IPython.display import display,HTML
sys.path.append('../python/')
#import train as nn
import random
import itertools
import scipy.stats as stats
#from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1450px !important; }</style>"))


style = {'description_width': 'initial'}

left_column='340px'
right_column='940px'
app_width='1282px'

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)
box_layout_image=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
    height='297px'
)
box_layout_phisical=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
)
box_layout_computing=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
)

box_layout_output=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    #height='150px'
)

form_item_layout_tab = Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    justify_content='space-between',
    padding='5px'
)

box_layout_tabs=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    height='100%'
)


button_hide_layout=Layout(
    visibility='hidden', #visible/hidden
    padding='5px',
    
)
button_show_layout=Layout(
    visibility='visible', #visible/hidden
    padding='5px',
    
)

hide_layout=Layout(
    visibility='hidden', #visible/hidden
    
)
show_layout=Layout(
    visibility='visible', #visible/hidden   
)


In [ ]:
'''
CSS = """
.output {
    align-items: center;
}
div.output_area {
    width: 100%;
}
div.output_area {
    left-padding: 60px;
}
"""
display(HTML('<style>{}</style>'.format(CSS)))

'''

js_custom="""<script>
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.
</script>
"""

display(HTML(js_custom))

In [ ]:
%%capture
# This hides any output from this cell
%set_env OMP_NUM_THREADS=4
os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 175px
}
</style>

In [ ]:
salt = widgets.BoundedFloatText(
    value=0.5,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Salt concentration (M):',
    style=style
)

In [ ]:
#options={'1 (monovalent)': 1, '2 (divalent)': 2, '3 (trivalent)': 3},
pion = widgets.Dropdown(
    options={'1 (monovalent)': 1, '2 (divalent)': 2, '3 (trivalent)': 3},
    value=1,
    description="Positive ion valency (e)",
    style=style
)


In [ ]:
nion = widgets.Dropdown(
    options=[-1],
    value=-1,
    description="Negative ion valency (e)",
    style=style
)


In [ ]:
clen = widgets.BoundedFloatText(
    value=3,
    min=3,
    max=5,
    step=0.1,
    description='Confinement length (nm):',
    style=style
)


In [ ]:
ion_diam_pos = widgets.BoundedFloatText(
    value=0.627,
    min=.2,
    max=.63,
    step=0.001,
    description='Positive ion diameter (nm):',
    style=style
)


In [ ]:
ion_diam_neg = widgets.BoundedFloatText(
    value=0.627,
    min=.2,
    max=.63,
    step=0.001,
    description='Negative ion diameter (nm):',
    style=style
)

# binding ion diameters together
dld2_link = widgets.jslink((ion_diam_pos, 'value'), (ion_diam_neg, 'value'))

In [ ]:
charge_on_wall = widgets.BoundedFloatText(
    value=0.0,
    min=-0.02,
    max=.0,
    step=0.005,
    description='Surface charge density (C/m^2):',
    style=style
)


In [ ]:
steps = widgets.BoundedIntText(
    value=10000,
    min=10000,
    max=5000000,
    step=10000,
    description='Simulation steps:',
    style=style
)

In [ ]:
imageSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=10000,
    description='Images',
    style=style
)

In [ ]:
%%capture
# This hides any output from this cell
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
plt.style.use('default')
import sklearn
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import joblib
#joblib.dump(scaler, 'infiles/scaler.pkl')

scaler = joblib.load('infiles/scaler.pkl')
'''
df = pd.read_csv('infiles/collectDatawhole.dat', delimiter = ",")
x_data = df.loc[:, '#Z_val':'d_val']
xbin = df.loc[:, 'stringPosition0':'stringPosition148']
y_label = df.loc[:, 'stringDensity0' : 'stringErrorBar148']
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(x_data)
'''
new_model = keras.models.load_model('infiles/my_model.h5')

In [ ]:
# new_model = keras.models.load_model('my_model.h5')
#new_model.summary()

In [ ]:
build_Plot = True

def display_preUI():
    with prediction_graph:
            display(figure_density_pre, layout=tab_layout) 

def predict_plot():
            
    global z_val, p_val, n_val, c_val, d_val, build_Plot, build_Plots, build_Plot_UQ
    global selected_pred, bins_selected
    
    if build_Plot:
        display_preUI()
        build_Plot = False
        
    if build_Plots:
        showPlotUI()
        build_Plots = False
        
    if build_Plot_UQ:
        showPlotUIUQ()
        build_Plot_UQ = False
    
    z_val = clen.value
    p_val = pion.value
    n_val = nion.value
    c_val = salt.value
    d_val = ion_diam_pos.value
    
    input_para = np.array([z_val, p_val, n_val, c_val, d_val])
    input_NN = scaler.transform(input_para.reshape(1,5))
    
    prediction = new_model.predict(input_NN)
    
    training_bin_width =0.04
    training_unitlength=0.5*d_val
    actual_bin_width = training_bin_width*training_unitlength
    
    bins_neg = np.arange(start=(-z_val/2.0 + actual_bin_width/2), stop=0.0, step=actual_bin_width)
    # inserting contact density point
    bins_neg = np.append(bins_neg, -z_val/2.0 + training_unitlength)
    bins_neg.sort()
 
    selected_pred = prediction.reshape(-1)[0:bins_neg.shape[0]]
    bins_selected = bins_neg[0:selected_pred.shape[0]]
    
    np.savetxt('data/ml_predicted_p_density_profile.dat', np.column_stack((bins_selected,selected_pred)), delimiter='\t')
    
    postive_den_download_ML.layout = button_show_layout
    
    plt.figure('prediction graph')
    plt.plot(bins_selected, selected_pred, c='g')
    plt.title('ML Predicted distribution of positive ions confined within the nanoparticle surfaces')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Density (M)', fontsize=10)
    plt.show()
    

###----------------------------------------------------------------------------------

def getUIInputs():
    salt_value = salt.value
    pion_value = pion.value
    nion_value = nion.value
    clen_value = clen.value
    ion_diam_value_pos = ion_diam_pos.value
    ion_diam_value_neg = ion_diam_neg.value
    charge_on_wall_value = charge_on_wall.value
    return salt_value, pion_value, nion_value, clen_value, ion_diam_value_pos, ion_diam_value_neg, charge_on_wall_value

def validateParamsMLSurrogate1():
    dic_surrogate1 = {"c":[0.3, 0.9],
                      "p":[1, 3],
                      "n":[-1, -1],
                      "z":[3.0, 4.0],
                      "dp":[0.5, 0.75],
                      "rho": [0.0, 0.0]
                     }
    
    salt_value, pion_value, nion_value, clen_value, ion_diam_value_pos, ion_diam_value_neg, charge_on_wall_value = getUIInputs()

    if (dic_surrogate1["c"][0] <= salt_value and dic_surrogate1["c"][1] >= salt_value) and (dic_surrogate1["p"][0] <= pion_value and dic_surrogate1["p"][1] >= pion_value) and (dic_surrogate1["n"][0] <= nion_value and dic_surrogate1["n"][1] >= nion_value) and (dic_surrogate1["z"][0] <= clen_value and dic_surrogate1["z"][1] >= clen_value) and (dic_surrogate1["dp"][0] <= ion_diam_value_pos and dic_surrogate1["dp"][1] >= ion_diam_value_pos) and (dic_surrogate1["rho"][0] <= charge_on_wall_value and dic_surrogate1["rho"][1] >= charge_on_wall_value) and ion_diam_value_pos==ion_diam_value_neg:
        return True
    else:
        return False
    

def onTogglePrediction(b):
    if predictionBtn.value:
        predictionBtn.icon = 'check' 
        predictionBtn.button_style = 'success'
        
        if validateParamsMLSurrogate1():
            predict_plot()
            postive_den_download_ML.layout = button_show_layout
        else:
            size_asym_warning_text.layout.visibility = 'visible'
            
        
    else:
        predictionBtn.icon = '' 
        predictionBtn.button_style=''
        postive_den_download_ML.layout = button_hide_layout
        size_asym_warning_text.layout.visibility = 'hidden'
        plt.figure('prediction graph')
        plt.clf()


predictionBtn = widgets.ToggleButton(
    value=False,
    description='Predict using ML',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Density profile prediction using ML is enabled when this button is turned on',
    icon='',
    style=style
)

predictionBtn.observe(onTogglePrediction,'value')

In [ ]:
submit_str = ''
def onToggleBtnChange(b):
    global submit_str
    if clustorMode.value:
        clustorMode.icon = 'check' 
        clustorMode.button_style = 'success'
        if not submit_str:
            output_image_warning_text.layout.visibility = 'visible'
    else:
        clustorMode.icon = '' 
        clustorMode.button_style=''
        if not submit_str:
            output_image_warning_text.layout.visibility = 'hidden'

'''
clustorMode =widgets.ToggleButton(
    value=False,
    description='Cluster mode',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on',
    icon='',
    style=style
)
'''

clustorMode =widgets.Checkbox(
    value=False,
    description='Cluster mode',
    disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on but you will loose the access to dynamic simulation snapshot slider.',
    icon='',
    style=style
)

clustorMode.observe(onToggleBtnChange,'value')

##----------------------------------------- 

def onToggleSizeAsymmetryBtn(b):
    global dld2_link
    if sizeAsymmetryBtn.value:
        sizeAsymmetryBtn.icon = 'check' 
        sizeAsymmetryBtn.button_style = 'success'
        dld2_link.unlink()

    else:
        sizeAsymmetryBtn.icon = '' 
        sizeAsymmetryBtn.button_style=''
        #dld2_link.link()
        dld2_link = widgets.jslink((ion_diam_pos, 'value'), (ion_diam_neg, 'value'))
        

sizeAsymmetryBtn = widgets.ToggleButton(
    value=False,
    description='Turn on size asymmetry',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Size asymmetry is enabled when this button is turned on',
    icon='',
    style=style
)

sizeAsymmetryBtn.observe(onToggleSizeAsymmetryBtn,'value')

In [ ]:
play_trigger = False

def play_btn_thread():
    global play_trigger
    imageSlider.value = 0
    while play_trigger:
            imageSlider.value = (imageSlider.value + imageSlider.step) % (imageSlider.max + imageSlider.step)
            time.sleep(1)
    
def onPlayBtnChange(b):
    global play_trigger
    global submit_str
    if playMode.value:
        playMode.button_style = 'success'
        play_trigger = True
        t = Thread(target=play_btn_thread)
        t.start()
        
    else:
        imageSlider.value=0
        playMode.button_style = ''
        play_trigger = False
            

playMode =widgets.ToggleButton(
    value=False,
    description='Play snapshots',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation snapshots will be played as a gif when this button is turned on',
    icon='',
    style=style
)

playMode.observe(onPlayBtnChange,'value')

In [ ]:
label_style = "style='background-color: #81c4fd; font-size:500; padding: 2px'"
label_style2 = "style='font-size:350; padding: 2px; font-weight: bold; align: center'"
def make_label(name, label_style, desc="",):
    lval = '<p %s %s>%s</p>' % (desc, label_style, name)
    return widgets.HTML(lval)

plabel = make_label('Physical Parameters', label_style, '')
clabel = make_label('Computing Parameters', label_style, '')
olabel = make_label('Output Controllers', label_style, '')
imageSliderabout = make_label('Slide to navigate the simulation snapshots', label_style2, '')

In [ ]:
'''
image_html = widgets.HTML(
    value='<img src="../images/overview.jpg" style="width: 500px"/>',
)
'''
file = open("../images/overview.jpg", "rb")
image = file.read()
image=widgets.Image(
    value=image,
    format='jpg',
    width=320,
    height=297,
)

form_items = [
    Box([image], layout=form_item_layout)
]

mainImage = Box(form_items, layout=box_layout_image)

In [ ]:
form_items_p = [
    Box([plabel], layout=form_item_layout),
    Box([salt], layout=form_item_layout),
    Box([pion], layout=form_item_layout),
    Box([nion], layout=form_item_layout),
    Box([clen], layout=form_item_layout),
    Box([ion_diam_pos], layout=form_item_layout),
    Box([ion_diam_neg], layout=form_item_layout),
    Box([charge_on_wall], layout=form_item_layout),
    Box([predictionBtn, sizeAsymmetryBtn], layout=form_item_layout)
]

form_items_c = [
    Box([clabel], layout=form_item_layout),
    Box([steps], layout=form_item_layout),
    Box([clustorMode], layout=form_item_layout)
    
]

form_items_output = [
    Box([olabel], layout=form_item_layout),
    Box([imageSlider], layout=form_item_layout),
    Box([imageSliderabout], layout=form_item_layout)
]

physical_para = Box(form_items_p, layout=box_layout_phisical)
computing_para = Box(form_items_c, layout=box_layout_computing)
output_para = Box(form_items_output, layout=box_layout_output)

In [ ]:
about=widgets.Textarea(
    value='This app enables users to simulate ions confined between nanoparticle (NP) surfaces in aqueous media. Nanoparticles can be synthetic (such as gold NPs) or natural (e.g. proteins) and the length of confinement is of the order of nanometers. Example systems include ion channel proteins of the cell membrane, adsorbed ions near surfaces of porous electrodes, and ions confined by NPs and/or colloidal particles. NP surfaces are assumed to be unpolarizable and are modeled as planar interfaces considering the large size difference between the ions and the NPs. \nThe app facilitates investigations for a wide array of ionic and environmental parameters. Users can extract the ionic structure (density profile) and study its dependence on salt concentration (c), ion valency (z), and other physical attributes. \nUsers can explore interesting effects by changing the c parameter from 0.3 to 0.9 M. This increase in density leads to crowding of the channel (confinement) with a large number of ions. The effect of symmetry breaking caused by the surfaces is seen: to avoid being pushed by ions from both the sides, an ion prefers the interface over the central region (bulk). The app enables users to explore this effect of ion accumulation near the interface, and make a quantitative assessment of ionic structure in strong confinement.\nAnother rich avenue to explore is to tune the valency of positive ions (parameter z) from 1 to 3. A positively-charged multivalent ion (+3 Fe or +2 Ca) near an interface is pulled away from the interface by oppositely charged ions with a stronger force relative to the bulk where the symmetry allows for no preferred movement. Thus, stronger electrostatic interactions (as in the case of multivalent ions) tend to cause depletion of the ions from the interface. This app empowers users to investigate this depletion effect via accurate computation of the density profiles of ions. \nEffects of changing other physical attributes such as confinement length and ion size are also available for users to explore. We invite users to take an inside look at what happens to the self-assembly of ions in these nanoscale channels by investigating the interplay of electrostatic effects and steric (or entropic) effects caused due to confinement, and measuring associated density profiles. \nWhile we provide a default of 20000 steps for users to get started on exploring different features of the app by obtaining immediate results, converged results of smooth profiles are expected for around 1 million steps. Users are given the option of clicking on "Cluster mode" button for accessing supercomputing resources to lower the computing time (however, they must allow for queue wait times that may vary). Simulation of confined ions is performed using LAMMPS and pre- and post-processing is carried out using the in-house C++ code. \nThis app is integrated with a machine learning (ML) layer for positive density profile prediction. Users can enable ML layer with "Predict using ML" button for accessing the ML predicted positive ion density profile in the "Prediction Graph" tab.  The same prediction graph is also available as an overlay in the "Positive Ion Density" tab when the simulation run is completed.',
    placeholder='',
    layout={'height': '310px','width': '100%'},
    disabled=True,
    style = {'font_weight': '150px'}
)

form_about = [
    Box([about], layout=form_item_layout)
]

box_layout_about=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    #height = '300px'
)

about_text = Box(form_about, layout=box_layout_about)


In [ ]:
def clear_All():
    localOutput.value = ""
    submit_str= ''
    progressBar.value=0
    imageSlider.max=0
    imageSlider.value=0 
    plt.figure('Distribution of positive ions')
    plt.clf()  # clear old plot
    plt.figure('Distribution of negative ions')
    plt.clf()  # clear old plot
    plt.figure('Distribution of net charge density')
    plt.clf()  # clear old plot
    plt.figure('Distribution of integrated charge')
    plt.clf()  # clear old plot
    
#     plt.figure('prediction graph')
#     plt.clf()  # clear old plot
    playMode.value = False
    playMode.layout.visibility = 'hidden'
    postive_den_download_ML.layout = button_hide_layout
    
    movie_download.layout = postive_den_download.layout = negative_den_download.layout = postive_den_download_ML.layout = button_hide_layout
    
    output_image.value=imageEmptyBox
    
    if clustorMode.value:
        output_image_warning_text.layout.visibility = 'visible'
    else:
        output_image_warning_text.layout.visibility = 'hidden'
    

In [ ]:
def runPreprocessor(z_value, p_value, n_value, c_value, d_value, a_value, s_value, rho_value):
    preprocessingCMD = "./md_simulation_confined_ions " + "-Z "+str(z_value)+" -p "+str(p_value)+" -n "+str(n_value)+" -c "+str(c_value)+" -d "+str(d_value)+" -a "+str(a_value)+" -S "+str(s_value)+" -J true -j true -i "+str(rho_value)+" > preprocessor.log"
    
    try:
        !$preprocessingCMD
        
    except  Exception as e:
        sys.stderr.write('Error occured during the execution of preprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1)

In [ ]:
runFlag=False
working_dir=""
parameter_append=""
sim_bigin=False
build_Plots=True
simulation_params=""
simulationStepsToatal=0

def callExe(s):
    global runFlag, working_dir, parameter_append, sim_bigin, ionsplot, build_Plots, simulation_params, simulationStepsToatal
    global clen_value, pion_value, nion_value, salt_value, ion_diam_value_pos, ion_diam_value_neg, charge_on_wall_value, sim_steps_value, build_Plot
    global submit_str
    if build_Plots:
        showPlotUI()
        build_Plots = False
        
    if build_Plot:
        display_preUI()
        build_Plot = False
        
    clear_All()
    
    localOutput.value = localOutput.value + "Simulation begins...\n"    
    
    salt_value = salt.value
    pion_value = pion.value
    nion_value = nion.value
    clen_value = clen.value
    ion_diam_value_pos = ion_diam_pos.value
    ion_diam_value_neg = ion_diam_neg.value
    charge_on_wall_value = charge_on_wall.value
    sim_steps_value = steps.value
    walltime = 10
    wallTimeMAX=50
    maxNumSteps = steps.max
    
    total_processors = 64
    simulationStepsToatal = sim_steps_value
    
    walltime = walltime + int(wallTimeMAX*sim_steps_value/maxNumSteps)
    
    #Adding standard outputs to string to be sent to output widget callback function
    parameter_append= "Salt concentration is "+ str(salt_value)+"\n"
    parameter_append= parameter_append + "Positive valency is "+ str(pion_value)+"\n"
    parameter_append= parameter_append + "Negative valency is "+ str(nion_value)+"\n"
    parameter_append= parameter_append + "Confinement length is "+ str(clen_value)+"\n"
    parameter_append= parameter_append + "Positive ion diameter is "+ str(ion_diam_value_pos)+"\n"
    parameter_append= parameter_append + "Negative ion diameter is "+ str(ion_diam_value_neg)+"\n"
    parameter_append= parameter_append + "Surface charge density is "+ str(charge_on_wall_value)+"\n"
    parameter_append= parameter_append + "Simulation steps is "+ str(sim_steps_value)+"\n"
    
    if clustorMode.value:
        parameter_append= parameter_append + "Requested walltime is "+ str(walltime)+" (mins)\n"
        parameter_append= parameter_append + "Requested total processors are "+ str(total_processors)+"\n"
    
    simulation_params="_%.2f" % float(clen_value)+"_%d" % int(pion_value)+"_%d" % int(nion_value)+"_%.2f" % float(salt_value)+"_%.3f" % float(ion_diam_value_pos)+"_%.3f" % float(ion_diam_value_neg)+"_%.3f" % float(charge_on_wall_value)+"_%d" % int(sim_steps_value)

    shutil.rmtree('data',True)
    if not os.path.exists('data'):
        os.makedirs('data')

    shutil.rmtree('outfiles',True)
    if not os.path.exists('outfiles'): 
        os.makedirs('outfiles')
        
    localOutput.value = localOutput.value + "Preprocessing is running...\n"
    
    #preprocessing stage
    runPreprocessor(clen_value, pion_value, nion_value, salt_value, ion_diam_value_pos, ion_diam_value_neg, sim_steps_value, charge_on_wall_value)   
    
    localOutput.value = localOutput.value + "Preprocessing completed\n"
        
    localOutput.value = localOutput.value +  parameter_append
    
    if predictionBtn.value and validateParamsMLSurrogate1():
        localOutput.value = localOutput.value + "ML layer invoked and ML predicted positive ion density profile is in the prediction graph tab. \nThe same prediction graph is also available as an overlay in the positive ion density tab when the simulation run is completed\n"
    
    # forcing OpenMP thread size, comment when deploying
    #os.environ["OMP_NUM_THREADS"] = "4"
    #os.environ["OMP_NUM_THREADS"] = "2"

    runName='nanoconfine'
    
    #computing_clusters = ['standby@conte','standby@rice']
    computing_clusters = ['standby@rice']
    
    #cluster_name = computing_clusters[random.randint(0,1)]
    cluster_name = computing_clusters[0]
    
    submit_para_list = '--venue ' + str(cluster_name) + ' -w '+ str(walltime)+' -n '+str(total_processors) \
    + ' -N ' + '16 ' + ' --tailStdout 1000 ' + '--inputfile ' + 'data ' + 'nanoconfinement-r32 '
    
    
    prog_paraList = ' -Z ' + str(clen_value) + ' -p ' + str(pion_value) + ' -n ' + str(nion_value) + ' -c ' + str(salt_value) \
    + ' -d ' + str(ion_diam_value_pos)+ ' -a ' + str(ion_diam_value_neg) + ' -S ' + str(sim_steps_value) + ' -f ' + str(simulation_params) + ' -v ' + 'false'

    
    runFlag=True
    sim_bigin=True
    
    #starting a thread for on the fly density plot
    #t2 = Thread(target=iterative_density_plot)
    #t2.start()
    
    try:
        submit_str = ""
        
        if clustorMode.value:
            submit_str = '--venue standby@brown -n '+str(total_processors)+' -N 16'+' -w '+str(walltime)+' -e OMP_NUM_THREADS=1'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'--inputfile '+'data '+'lammps-31Mar17-parallel -in in.lammps'
            #submit_str = '-n '+str(total_processors)+' -w '+str(walltime)+' -N 20'+' -e OMP_NUM_THREADS=1'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps' 
            #submit_str = '--venue standby@rice ' + '-n 32 -w 60 -N 16'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps'
            localOutput.value = localOutput.value + "LAMMPS input script designed and simulation submitted to the cluster, waiting for response...\n"
        else:
            submit_str = '--local stdbuf -o0 -e0 lmp_serial -in in.lammps'
            localOutput.value = localOutput.value + "LAMMPS input script designed and simulation running...\n"
        #rname = s.make_rname(e_np_value, e_out_value, pion_value, c_out_value, np_radius_value, np_charge_value, sim_steps_value)
        #run_str = 'stdbuf -o0 -e0 lmp_serial -in in.lammps' 
        
        #submit_str = submit_para_list + prog_paraList
        #parameter_append= parameter_append + "Running the simulation on "+ str(cluster_name)+"\n"
        
        #submit_str = '--local pwd'
        s.run(runName, submit_str)
        
        time.sleep(1)
           
        
    except  Exception as e:
        sys.stderr.write('Error occured during the execution of md_simulation_confined_ions')
        sys.stderr.write(str(e))
        sys.exit(1)
    
    time.sleep(1)
    runFlag = False
    #t2.join()
  
 

In [ ]:
def runPostProcessor(clen_value, pion_value, nion_value, salt_value, ion_diam_value_pos, ion_diam_value_neg, sim_steps_value, rho_value):
    
    postprocessingCMD = "./md_simulation_confined_ions " + "-Z "+str(clen_value)+" -p "+str(pion_value)+" -n "+str(nion_value)+" -c "+str(salt_value)+" -d "+str(ion_diam_value_pos)+" -a "+str(ion_diam_value_neg)+" -S "+str(sim_steps_value)+" -J true -j false -i "+str(rho_value)+" > postprocessor.log"
    
    try:
        !$postprocessingCMD
        
    except  Exception as e:
        print(str(e))
        sys.stderr.write('Error occured during the execution of postprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1) 
        

In [ ]:
def read_lammps_density_profiles(path):
    header_len = 3
    header_colmns = 0
    data_len = 0
    data_colmns = 0
    z_val_col = 1
    density_col = 3
    time_steps = []
    all_data = []
    
    try:
        # if lammps header is there remove it
        with open(path, 'r') as fin:
            data = fin.read().splitlines(True)
            pointer = 0
            #for line in data:
            while pointer < len(data):
                
                if '# Chunk-averaged data for fix' in data[pointer]:
                    header_colmns = len(data[pointer+1].split())
                    data_colmns = len(data[pointer+2].split())
                    pointer += header_len
                    #print(header_colmns, data_colmns)
                
                # Read timestep header
                line = data[pointer].split()
                time_step = int(line[0])
                time_steps.append(time_step)
                data_len = int(line[1])
                #print(time_step, data_len)
                # moving pointer to actual data
                pointer += 1
                current_data_end = data_len + pointer
                data_samples = []
                
                while pointer < current_data_end:
                    line = data[pointer].split()
                    z_val = float(line[z_val_col])
                    density_val = float(line[density_col])
                    data_samples.append([z_val, density_val])
                    pointer += 1

                all_data.append(data_samples)
                    
        
        time_steps = np.array(time_steps)
        all_data = np.array(all_data)
        #print(time_steps.shape)
        #print(all_data.shape)
        #density_profile_p = np.loadtxt(path)
        return (time_steps, all_data)
    except Exception as e: 
        print(e)
        import traceback
        traceback.print_exc()
        sys.stderr.write('Can not find the positive density results file')
        sys.exit(1)

def save_density_profile_lmp_with_errobars(density_data, file_name):
    
    global clen_value, pion_value, nion_value, salt_value, ion_diam_value_pos, ion_diam_value_neg

    smaller_diameter = min(ion_diam_value_pos, ion_diam_value_neg)
    
    density = np.mean(density_data[1:,:,:], axis=0)
    density_errors = np.std(density_data[1:,:,1], axis=0)

    xx = density[:,0]*smaller_diameter
    yy = density[:,1]/smaller_diameter**3 / 0.6022
    yrr = density_errors/smaller_diameter**3 / 0.6022
   
    np.savetxt('data/'+ file_name, np.column_stack((xx, yy, yrr)), delimiter='\t')


def save_density_profiles_lmp(path=''):
    try:
        path_pos = path + 'data/p_density_profile.lmp.for.errorbars.dat'
        path_neg = path + 'data/n_density_profile.lmp.for.errorbars.dat'
        
        # if lammps header is there remove it
        _, density_profile_p = read_lammps_density_profiles(path_pos)
        #print(density_profile_p.shape)
        save_density_profile_lmp_with_errobars(density_profile_p, "p_density_profile.dat")
        
        _, density_profile_n = read_lammps_density_profiles(path_neg)
        save_density_profile_lmp_with_errobars(density_profile_n, "n_density_profile.dat")
        
    except Exception as e: 
        print(e)
        sys.stderr.write('Can not find the density results files from lammps')
        sys.exit(1) 

In [ ]:
def doneFunc(s, rdir):
    global submit_str, simulationStepsToatal  
    global oneTimePrintFlag
    submit_str = ""
    progressBar.value=int(100)
    playMode.layout.visibility = 'visible'
    
    salt.disabled = False
    pion.disabled = False
    nion.disabled = False
    clen.disabled = False
    ion_diam_pos.disabled = False
    ion_diam_neg.disabled = False
    charge_on_wall.disabled = False
    
    oneTimePrintFlag = True
    
    imageSlider.max=simulationStepsToatal - simulationStepsToatal%imageSlider.step
    imageSlider.value=simulationStepsToatal - simulationStepsToatal%imageSlider.step
    
    localOutput.value = localOutput.value + "LAMMPS simulation finished\n" 
    
    submit_btn.but.disabled = True
    
    localOutput.value = localOutput.value + "Postprocessing is running...\n"
    
    #postprocessing stage 
    #runPostProcessor(clen_value, pion_value, nion_value, salt_value, ion_diam_value_pos, ion_diam_value_neg, sim_steps_value, charge_on_wall_value)
    save_density_profiles_lmp()
        
    localOutput.value = localOutput.value + "Postprocessing is finished\n"
    
    #  Plotting the result
    if predictionBtn.value and validateParamsMLSurrogate1():
        plot_data_with_prediction()
    else:
        plot_data()
    
    localOutput.value = localOutput.value + "Simulation ended\n"
    
    submit_btn.but.disabled = False
    
    if predictionBtn.value and validateParamsMLSurrogate1():
        postive_den_download_ML.layout = button_show_layout
    else:
        postive_den_download_ML.layout = button_hide_layout
    
    movie_download.layout = postive_den_download.layout = negative_den_download.layout =  button_show_layout

    

In [ ]:
# after the run finished, optionally do something
# with the output
def plot_data():
    global build_Plots
    if build_Plots:
        showPlotUI()
        build_Plots = False    
    
    #We ignore rdir because submit puts non-parametric runs in the current directory
    # When caching is enabled, they get put in a chached directory and rdit is always used. 
    #wait for 2 seconds before plotting errorBars.
    time.sleep(2)
    plt.figure('Distribution of positive ions')
    plt.clf()  # clear old plot
    plt.title('Distribution of positive ions confined within the nanoparticle surfaces')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Density (M)', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        density_profile = np.loadtxt(working_dir + 'data/p_density_profile.dat')
    except:
        sys.stderr.write('Can not find the positive density results file')
        sys.exit(1)
    #print(density_profile)    
    x = density_profile[:,0]
    y = density_profile[:,1]
    yrr = density_profile[:,2]
    plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='r', capsize=2)
    plt.show()
    
    plt.figure('Distribution of negative ions')
    plt.clf()  # clear old plot
    plt.title('Distribution of negative ions confined within the nanoparticle surfaces')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Density (M)', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        ndensity_profile = np.loadtxt(working_dir + 'data/n_density_profile.dat')
    except:
        sys.stderr.write('Can not find the negative density results file')
        sys.exit(1)
    #print(ndensity_profile)    
    x = ndensity_profile[:,0]
    y = ndensity_profile[:,1]
    yrr = ndensity_profile[:,2]
    plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='r', capsize=2)
    
    plt.show()
    
    #Plot integrated charge
    plot_integrated_charge_data(density_profile, ndensity_profile)
    

def compute_screening_len(xx_half, yy_half, yyrr_half, allowed_percent):
    
    allowed_y = abs(yy_half[0])*allowed_percent
    allowed_noice_points=3
    allowed_good_points=50
    screening_len = None
    temp_screening_len = None
    noice_count = 0
    good_points_count = 0
    
    for x_value, y_value, y_error in zip(xx_half[::-1], yy_half[::-1], yyrr_half[::-1]):
        # This means current point is within 5%
        # If upperlimit or lower limit within the 5% tube, then that is valid
        #print(y_value+y_error, -allowed_y <= y_value+y_error <= allowed_y )
        #print(y_value-y_error, -allowed_y <= y_value-y_error <= allowed_y )
        if (-allowed_y <= y_value-y_error <= allowed_y) or (-allowed_y <= y_value+y_error <= allowed_y):
            noice_count = 0
            good_points_count += 1

        else:
            temp_screening_len = x_value
            noice_count += 1
        #print(x_value, noice_count, good_points_count)        
        if noice_count >= allowed_noice_points and good_points_count >= allowed_good_points:
            screening_len = temp_screening_len
            break
    
    return screening_len

def get_screening_len(xx_half, yy_half, yyrr_half):
    allowed_percents = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
    screening_lens = []
    #print(-allowed_y, allowed_y)
    for allowed_percent in allowed_percents:
        screening_len_ = compute_screening_len(xx_half, yy_half, yyrr_half, allowed_percent)
        if screening_len_ is not None:
            screening_lens.append(screening_len_)

    screening_lens = np.array(screening_lens)
    if screening_lens.shape[0]>0:
        screening_len = np.mean(screening_lens)
    else:
        screening_len = None 
        
    return screening_len    
    
def plot_integrated_charge_data(pdensity_profile, ndensity_profile):
    global clen_value, charge_on_wall_value
    global oneTimePrintFlag
    
    x = pdensity_profile[:,0]
    y = pdensity_profile[:,1] - ndensity_profile[:,1]
    yrr = np.sqrt(pdensity_profile[:,2]**2 + ndensity_profile[:,2]**2)
    
    #Distribution of net charge density
    plt.figure('Distribution of net charge density')
    plt.clf()  # clear old plot
    plt.title('Distribution of net charge density for confined electrolytes')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Net charge density $\\rho$ (eM)', fontsize=10)
    plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='r', capsize=2)
    plt.show()
    
    time.sleep(0.5)
    #------------------------------------------------
    #screenfactor_at_z_position = screenfactor_at_z_position + FaradayConstant * ((p_density_at_z_position - n_density_at_z_position) * (bin_width * unitlength * pow(10, -9)) * (1000 / abs(charge_density)));	// because you demand to get netdensity in M
    bin_width_nm =  abs(pdensity_profile[0,0] - pdensity_profile[1,0])
    
    xx = pdensity_profile[:,0] + clen_value/2
    
    # integrated charge
    yy_temp = (pdensity_profile[:,1] - ndensity_profile[:,1])*bin_width_nm*0.096
    yy = []
    charge_sum = charge_on_wall_value
    for yy_temp_val in yy_temp:
        charge_sum += yy_temp_val
        yy.append(charge_sum)  
    yy = np.array(yy)
    yyrr = np.sqrt(pdensity_profile[:,2]**2 + ndensity_profile[:,2]**2)*bin_width_nm*0.096
    
    #Take half of the integrated charge profile
    xx_half = xx[0:xx.shape[0]//2:1]
    yy_half = yy[0:xx.shape[0]//2:1]
    yyrr_half = yyrr[0:xx.shape[0]//2:1]
    
    plt.figure('Distribution of integrated charge')
    plt.clf()  # clear old plot
    plt.title('Distribution of integrated charge of the electrolyte system')
    plt.xlabel('Distance from left surface z + h/2 (nm)', fontsize=10)
    plt.ylabel('Integrated charge S (C/m$^2$)', fontsize=10)
    
    plt.errorbar(xx_half,yy_half,xerr=0.0, yerr=yyrr_half, c='b', ecolor='r', capsize=2)
    
    #------------------------------------------ Screening lenth
    allowed_percent = 0.05
    allowed_y = abs(yy[0])*allowed_percent
    
    plt.axhline(y=allowed_y, color = "green", alpha=0.2, linestyle='--')
    plt.axhline(y=-allowed_y, color = "green", alpha=0.2, linestyle='--')
    
    screening_len = get_screening_len(xx_half, yy_half, yyrr_half)

    if screening_len is not None:
        if oneTimePrintFlag:
            localOutput.value = localOutput.value + "Screening lenth is {:.4f} nm\n".format(screening_len)
            oneTimePrintFlag=False
        plt.axvline(x=screening_len, color = "black", alpha=0.4, linestyle='--')
        plt.text(screening_len, allowed_y, "Screening lenth ={:.4f}".format(screening_len), ha='center', va='bottom',rotation='vertical', backgroundcolor='white')
    else:
        if oneTimePrintFlag:
            localOutput.value = localOutput.value + "Screening lenth could not be found with the generated statistics \n"
            oneTimePrintFlag=False
    
    plt.show()


In [ ]:
def plot_data_with_prediction():
    
    global build_Plots
        
    if build_Plots:
        showPlotUI()
        build_Plots = False
    
    #We ignore rdir because submit puts non-parametric runs in the current directory
    # When caching is enabled, they get put in a chached directory and rdit is always used. 
    #wait for 2 seconds before plotting errorBars.
    time.sleep(2)
    plt.figure('Distribution of positive ions')
    plt.clf()  # clear old plot
    plt.title('Distribution of positive ions confined within the nanoparticle surfaces')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Density (M)', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        density_profile = np.loadtxt(working_dir + 'data/p_density_profile.dat')
    except:
        sys.stderr.write('Can not find the positive density results file')
        sys.exit(1)
    #print(density_profile)    
    x = density_profile[:,0]
    y = density_profile[:,1]
    yrr = density_profile[:,2]
    plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='m', capsize=2)
    
    np.savetxt('data/ml_predicted_p_density_profile.dat', np.column_stack((bins_selected,selected_pred)), delimiter='\t')
    plt.plot(bins_selected, selected_pred, c='g')
    plt.legend([ 'ML prediction','MD simulation'], loc='upper left')
    plt.show()
    
    plt.figure('Distribution of negative ions')
    plt.clf()  # clear old plot
    plt.title('Distribution of negative ions confined within the nanoparticle surfaces')
    plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=10)
    plt.ylabel('Density (M)', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        ndensity_profile = np.loadtxt(working_dir + 'data/n_density_profile.dat')
    except:
        sys.stderr.write('Can not find the negative density results file')
        sys.exit(1)
    #print(ndensity_profile)    
    x = ndensity_profile[:,0]
    y = ndensity_profile[:,1]
    yrr = ndensity_profile[:,2]
    plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='m', capsize=2)
    
    plt.show()
    
    #Plot integrated charge
    plot_integrated_charge_data(density_profile, ndensity_profile)

In [ ]:
in_wrapper_progress = False
saved_data = ""
saved_data_enery = ""
saved_data_ions_pos = ""
stepNumber = 0
def stdoutput_handler(buffer):
    global submit_str
    global in_wrapper_progress, saved_data, parameter_append, sim_bigin, stepNumber, saved_data_enery, saved_data_ions_pos
    output = []
    #return buffer
    
    if sim_bigin:
        #output.append(parameter_append)
        sim_bigin = False
   
    data = buffer.split('\n')
    last = len(data) - 1
    
    for i, line in enumerate(data):
        output.append(line)
        #print(line)
        number = line.split()
        if len(number)>10:
            if number[0].isdigit():
                #print(number[0])
                percent = float(number[0])/simulationStepsToatal*100
                progressBar.value=int(percent)
                #print(percent)
                if submit_str.startswith('--local '):
                    if(int(number[0])%imageSlider.step == 0):
                        imageSlider.max=number[0]
                        imageSlider.value=number[0]
        
        '''
        if line.startswith('Loop time of '):
            in_wrapper_progress = False
            continue
        
        if in_wrapper_progress:
            #print(line)
            number = line.split()
            if number:
                if number[0].isdigit():
                    print(number[0])
                    percent = float(number[0])/simulationStepsToatal*100
                    progressBar.value=int(percent)
                    #print(percent)
                    if submit_str.startswith('--local '):
                        if(int(number[0])%imageSlider.step == 0):
                            imageSlider.max=number[0]
                            imageSlider.value=number[0]
            continue
        
        if line.startswith('Step Temp Volume Density KinEng PotEng TotEng'):
            #print(line)
            in_wrapper_progress = True
            
        '''    
    reduced_buffer = '\n'.join(output)
    
    #appending new line at the end
    if reduced_buffer:
        if reduced_buffer[-1] != '\n':
            reduced_buffer = reduced_buffer + '\n'
    
    return reduced_buffer

In [ ]:
#button_style = "style='background-color: #81c4fd; font-size:500; padding: 4px; -webkit-appearance: button; -moz-appearance: button; appearance: button; text-decoration: none; color: initial;'"

def make_downloadBtn(downFilename="", downloadURL="", displayText=""):
    bval = "<a download='%s' href='%s'><button class='p-Widget jupyter-widgets jupyter-button widget-button mod-info'>%s</button></a>" % (downFilename, downloadURL, displayText)
    return widgets.HTML(bval)

#from IPython.display import FileLink, FileLinks
#FileLinks('outfiles')
#FileLinks('data')

In [ ]:
%%capture
# This hides any output from this cell  

tab_layout=Layout(
    #width='100%',
    height='643.5px',
    justify_content='space-between',
)

# we want the plot to go here, so create an empty one initially
#create tabs
figsize_tup=(8.60, 5.60)
plt.ioff() #LINE ADDED, Interactive plot blur fix
figure_density_pos = plt.figure(num='Distribution of positive ions', figsize=figsize_tup)
out_desnity_pos = Output(layout=tab_layout)

figure_density_neg = plt.figure(num='Distribution of negative ions', figsize=figsize_tup)
out_desnity_neg = Output(layout=tab_layout)

figure_net_charge_density = plt.figure(num='Distribution of net charge density', figsize=figsize_tup)
out_net_charge_density = Output(layout=tab_layout)

figure_integrated_charge = plt.figure(num='Distribution of integrated charge', figsize=figsize_tup)
out_desnity_integrated_charge = Output(layout=tab_layout)


#------------------------------- Asym notice -------------------------------

tab_layout_size_asym = Layout(
    #width='100%',
    height='643.5px',
    justify_content='space-between',
    flex_flow='column',
)

output_prediction_graph_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    margin = '0px 0px 0px 0px', #top, right, bottom and left
    #height='80%',
    width='100%',
)


box_layout_size_asym=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 1px',
    align_items='stretch',
    #padding='5px',
    margin = '0px 0px 0px 25px',
    width='100%',
    visibility = 'hidden',
    
)


box_layout_for_right_size_asym = Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    #border='solid',
    #height='100%',
    width='100%',
    #height='20%'
    #width='150px',
    #width='50%'
)

figsize_tup_asym=(8.60, 5.30)

figure_density_pre = plt.figure(num='prediction graph', figsize=figsize_tup_asym)
prediction_graph = Output(layout=output_prediction_graph_layout)
plt.ion() #LINE ADDED

#label_style3 = "style='background-color: #eeeeee; color: #388e3c; font-size:350; padding: 2px; font-weight: bold; align: center'"
label_style_size_asym = "style='background-color: #eeeeee; font-size:350; padding: 2px; font-weight: bold; align: center'"
size_asym_warning_text = make_label('Surrogate has not learned ion size asymmetry effects yet. Coming soon! Turn off size asymmetry button to enable ML predictions.', label_style_size_asym, '')


form_size_asym_warning_text = [
    Box([size_asym_warning_text], layout=box_layout_size_asym)
]

form_size_asym_warning_text_sec_right = Box(form_size_asym_warning_text, layout=box_layout_for_right_size_asym)

prediction_graph_box =  Box([prediction_graph], layout=output_prediction_graph_layout)

form_output_prediction_sec = [
    Box([form_size_asym_warning_text_sec_right], layout=box_layout_for_right_size_asym),
    Box([prediction_graph_box], layout=output_prediction_graph_layout)
]
output_prediction_graph_section = Box(form_output_prediction_sec, layout=tab_layout_size_asym)
#size_asym_warning_text.layout.visibility = 'visible'
#----------------------------------------------------------------------------


file = open("../images/image.emptybox.jpg", "rb")
imageEmptyBox = file.read()

output_image=widgets.Image(
    format='jpg',
    width=615,
    #height=650,
    value=imageEmptyBox
)

#figure_density_both = plt.figure(num='Distribution of positive/negative ions', figsize=figsize_tup)
#out_desnity_both = Output(layout=tab_layout)

output_image_layout=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    margin = '0px 0px 0px 50px' #top, right, bottom and left
)

box_layout_warning_text=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 1px',
    align_items='stretch',
    padding='5px',
    width='150px',
    visibility = 'hidden',
    height='95%'
)
box_layout_playbtn=Layout(
    display='flex-end',
    flex_flow='row',
    #border='solid 1px',
    align_items='stretch',
    padding='5px',
    width='150px',
    visibility = 'hidden',
    #height='136px'
)


#label_style3 = "style='background-color: #eeeeee; color: #388e3c; font-size:350; padding: 2px; font-weight: bold; align: center'"
label_style3 = "style='background-color: #eeeeee; font-size:350; padding: 2px; font-weight: bold; align: center'"
output_image_warning_text = make_label('Cluster mode is on! You will lose access to the dynamic simulation snapshot data. Snapshot data will be available at the end of simulation.', label_style3, '')

box_layout_for_right = Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    #border='solid',
    height='100%',
    #width='150px',
    #width='50%'
)

form_output_image_sec_right_items = [
    Box([output_image_warning_text], layout=box_layout_warning_text),
    Box([playMode], layout=box_layout_playbtn),
]

form_output_image_sec_right = Box(form_output_image_sec_right_items, layout=box_layout_for_right)

output_image_box =  Box([output_image], layout=output_image_layout)

form_output_image_sec = [
    Box([output_image_box], layout=output_image_layout),
    Box([form_output_image_sec_right], layout=box_layout_for_right),
]
output_image_section = Box(form_output_image_sec, layout=tab_layout)


#download button
movie_download = make_downloadBtn(downFilename="electrolyte_movie.xyz", downloadURL="outfiles/electrolyte_movie.xyz", displayText="Movie")
postive_den_download = make_downloadBtn(downFilename="p_density_profile.dat", downloadURL="data/p_density_profile.dat", displayText="Positive ion density")
postive_den_download_ML = make_downloadBtn(downFilename="ml_predicted_p_density_profile.dat", downloadURL="data/ml_predicted_p_density_profile.dat", displayText="ML prediction")
negative_den_download = make_downloadBtn(downFilename="n_density_profile.dat", downloadURL="data/n_density_profile.dat", displayText="Negative ion density")

movie_download.layout = postive_den_download.layout = negative_den_download.layout = postive_den_download_ML.layout =  button_hide_layout

box_layout_outputs=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    padding='5px'
)

form_output_content = [
    Box([movie_download], layout=box_layout_outputs),
    Box([postive_den_download], layout=box_layout_outputs),
    Box([postive_den_download_ML], layout=box_layout_outputs),
    Box([negative_den_download], layout=box_layout_outputs)
]
downloadMovieSec = Box(form_output_content, layout=tab_layout)

## UQ_Logic

In [ ]:
%%capture

def figure_clear_up_uq():
    plt.figure('Salt concentration')
    plt.clf()  # clear old plot
    plt.figure('Ion diameter')
    plt.clf()  # clear old plot
    plt.figure('Positive Ion Density Distribution at Z')
    plt.clf()
    figure_output_sens_uq__ = plt.figure('Sensitivity of Positive Ion Density at Z')
    ax_sens_uq__ = figure_output_sens_uq__.axes
    ax_sens_uq__[0].cla()
    ax_sens_uq__[1].cla()
    #ax_sens_uq__[2].cla() 
    plt.figure('Distribution of Ion Density for UQ')
    plt.clf()



def click_simulate_uq(b):
    
    # If surrogate is not valid for the defined input parameter regions, no UQ
    if not validateParamsMLSurrogate1():
        UQ_localOutput.value += "Input parameter ranges are not valid for the existing ML surrogate.\n"
        return 
        
    
    global build_Plot_UQ, contact_point, selected_pred_uq, bins_selected_uq, input_list_uq, paramlist
    if simulate_uq.description == 'Simulate':
        #################################################################
        UQ_localOutput.value = ""
        simulate_uq.disabled=True
        progressBar_uq.value=0
        #GUI showcasing
        if build_Plot_UQ:
            UQ_localOutput.value += "Buiding GUI for plotting.\n"
            showPlotUIUQ()
            build_Plot_UQ = False
        else:
            UQ_localOutput.value += "Clearing GUI for new plotting.\n"
            figure_clear_up_uq()   
                      
        #################################################################
        #Build the input data
        UQ_localOutput.value += "Random sampling inputs.\n"
        salt_value = salt.value
        pion_value = pion.value
        nion_value = nion.value
        clen_value = clen.value
        ion_diam_value_pos = ion_diam_pos.value
        ion_diam_value_neg = ion_diam_neg.value
        #sim_steps_value = steps.value
        
        salt_dsit_value = salt_uq_select_drop.value
        salt_dsit_value1 = salt_uq_1.value
        salt_dsit_value2 = salt_uq_2.value
        ion_dsit_value = ion_diam_uq_select_drop.value
        ion_dsit_value1 = ion_diam_uq_1.value
        ion_dsit_value2 = ion_diam_uq_2.value
        
        number_of_samples_uq = samples_per_parameter.value
        
        if salt_dsit_value== 'gaussian':
            salt_value = np.random.normal(salt_dsit_value1, salt_dsit_value2, number_of_samples_uq)
        elif salt_dsit_value== 'uniform':
            salt_value = np.random.uniform(salt_dsit_value1,salt_dsit_value2,number_of_samples_uq)
        elif salt_dsit_value== 'exact':
            salt_value=salt_dsit_value1
            
        if ion_dsit_value== 'gaussian':
            ion_diam_value_neg = ion_diam_value_pos = np.random.normal(ion_dsit_value1, ion_dsit_value2, number_of_samples_uq)
        elif ion_dsit_value== 'uniform':
            ion_diam_value_neg = ion_diam_value_pos = np.random.uniform(ion_dsit_value1,ion_dsit_value2,number_of_samples_uq)
        elif ion_dsit_value== 'exact':
            ion_diam_value_neg = ion_diam_value_pos=ion_dsit_value1
            
        UQ_localOutput.value += "Plotting input distributions.\n"
        
        plt.figure('Salt concentration')
        plt.clf()  # clear old plot
        #plt.title('Distribution of negative ions confined within the nanoparticle surfaces')
        plt.xlabel('Salt concentration (M)', fontsize=9)
        plt.ylabel('Probability Density (PD)', fontsize=9)
        #range=(0,1)
        count, bins, ignored = plt.hist(salt_value, bins=50, density=True, alpha=0.8, ec="black", color='C0')
        #print(vars(plt))#, color='blue'
        plt.tight_layout()
        plt.figure('Ion diameter')
        plt.clf()  # clear old plot
        #plt.title('Distribution of negative ions confined within the nanoparticle surfaces')
        plt.xlabel('Ion diameter (nm)', fontsize=9)
        plt.ylabel('Probability Density (PD)', fontsize=9)
        #range=(0,1)
        count, bins, ignored = plt.hist(ion_diam_value_pos, bins=50, density=True, alpha=0.8, ec="black", color='magenta')
        plt.tight_layout()
        
        
        if salt_dsit_value== 'exact':
            salt_value= [salt_value]
        if ion_dsit_value== 'exact':
            ion_diam_value_neg= [ion_diam_value_neg]
        
        UQ_localOutput.value += "Buiding bag of jobs for all input combinations.\n"
        
        input_list_uq = ([clen_value], [pion_value],[nion_value], salt_value, ion_diam_value_neg)
        
        paramlist = list(itertools.product(*input_list_uq))
        #print(len(input_list_uq))
        
        UQ_localOutput.value += "Starting ML surrogate to do " +str(len(paramlist)) +" predictions...\n"
        
        paramlist = np.array(paramlist)
        paramlist = scaler.transform(paramlist.reshape(-1,5))
        
        prediction=[]
        ## Loop to run 100,000 at once
        for i in range(0,len(paramlist),10000):
            if i==0:
                prediction=new_model.predict(paramlist[i:i+10000,:])
            else:
                prediction = np.append(prediction,new_model.predict(paramlist[i:i+10000,:]) ,axis=0)
            progressBar_uq.value= int((i+10000)/len(paramlist)*100.0)
            
        #print(paramlist.shape)
        #prediction = new_model.predict(paramlist)
        #print(prediction.shape)
        
        UQ_localOutput.value += "All predictions are completed.\n"
        
        training_bin_width =0.04
        training_unitlength=0.5*ion_dsit_value1
        actual_bin_width = training_bin_width*training_unitlength
        
        bins_neg = np.arange(start=(-clen_value/2.0 + actual_bin_width/2), stop=0.0, step=actual_bin_width)
        # inserting contact density point
        contact_point = -clen_value/2.0 + training_unitlength
        bins_neg = np.append(bins_neg, contact_point)
        bins_neg.sort()
        
        selected_pred_uq = prediction[:,0:bins_neg.shape[0]]
        bins_selected_uq = bins_neg[0:selected_pred_uq.shape[1]]
        
        contact_point_index = np.where(bins_selected_uq == contact_point)[0][0]
        
        imageSlider_uq_output.min=bins_selected_uq[0]
        imageSlider_uq_output.max=bins_selected_uq[-1]
        imageSlider_uq_output.step=actual_bin_width
        imageSlider_uq_output.value=bins_selected_uq[0]  # this allows refresh if rerun with same diameter
        imageSlider_uq_output.value=bins_selected_uq[contact_point_index]
        
        imageSlider_uq_text.layout = show_layout
        imageSlider_uq_output.layout = show_layout
        uq_contact_button.layout = show_layout
        output_uq_mean.layout = show_layout_btn_output
        output_uq_var.layout = show_layout_btn_output
        output_uq_std.layout = show_layout_btn_output
        out_uq_dist_inf.layout = show_layout 
        #UQ_localOutputWidget.layout.visibility='visible'
        sens_salt_label_inf.layout = show_layout
        sens_salt_uq_mean.layout = show_layout_btn_uq
        sens_salt_uq_std.layout = show_layout_btn_uq
        sens_diam_label_inf.layout = show_layout
        sens_diam_uq_mean.layout = show_layout_btn_uq
        sens_diam_uq_std.layout = show_layout_btn_uq
    
        plt.figure('Distribution of Ion Density for UQ')
        plt.clf()
        plt.xlabel('Distance between the two surfaces (z = 0 is the midpoint) (nm)', fontsize=9)
        plt.ylabel('Density (M)', fontsize=9) 
        
        y__ = selected_pred_uq[np.random.choice(range(selected_pred_uq.shape[0]), min(selected_pred_uq.shape[0], 10000), replace=False),:]
        
        h = plt.hist2d(np.tile(bins_selected_uq, y__.shape[0]), y__.reshape(-1), bins=(y__.shape[1], 50),  normed=True)
        #plt.hist2d(x, y, bins=40, norm=LogNorm())
        cbar = plt.colorbar(h[3])
        cbar.set_label('Probability Density (PD)', fontsize=9)#, rotation=270
        plt.tight_layout()
        
        simulate_uq.disabled=False

    else:
        pass
    
    
def onUQ_SliderChange(b):
    global selected_pred_uq, bins_selected_uq, input_list_uq, paramlist

    value_bin_uq = imageSlider_uq_output.value
    
    selected_point_index = (np.abs(bins_selected_uq - value_bin_uq)).argmin()

    plt.figure('Positive Ion Density Distribution at Z')
    plt.clf()
    #time.sleep(2)
    plt.xlabel('Density (M)', fontsize=9)
    plt.ylabel('Probability Density (PD)', fontsize=9)
    #range=(0,1)
    count, bins, ignored = plt.hist(selected_pred_uq[:,selected_point_index], bins=50, density=True, alpha=0.8, color = "green", ec="black")
    
    mean_uq_ = np.mean(selected_pred_uq[:,selected_point_index])
    std_uq_ = np.std(selected_pred_uq[:,selected_point_index])
    var_uq_ = np.var(selected_pred_uq[:,selected_point_index])
    output_uq_mean.value=mean_uq_
    output_uq_var.value=var_uq_
    output_uq_std.value=std_uq_ 
    
    #plot a gauusian with mu and sigma
    x_uq = np.linspace(mean_uq_ - 3*std_uq_, mean_uq_ + 3*std_uq_, 100)
    plt.plot(x_uq, stats.norm.pdf(x_uq, mean_uq_, std_uq_), color = "blue", alpha=0.4, linestyle='-')
    
    plt.tight_layout()
    
    ## Sensitivity
    #paramlist_subsampled= np.array(paramlist).reshape(len(input_list_uq[3]),len(input_list_uq[4]), len(input_list_uq))
    #reshaping to get n*m
    selected_pred_uq_subsampled= np.array(selected_pred_uq[:,selected_point_index]).reshape(len(input_list_uq[3]),len(input_list_uq[4]))
    
    #average over n to keep salt const. And get a average over all those salt value while varying diameter
    dimeter_sense_data_out = selected_pred_uq_subsampled.reshape(len(input_list_uq[3]),-1)
    dimeter_sense_data_exp = np.mean(dimeter_sense_data_out, axis=1)
    dimeter_sense_data_var = np.mean((dimeter_sense_data_exp-mean_uq_)**2)
    #wise versa
    salt_sense_data_out = np.transpose(selected_pred_uq_subsampled, (1,0)).reshape(len(input_list_uq[4]),-1)
    salt_sense_data_exp = np.mean(salt_sense_data_out, axis=1)
    salt_sense_data_var = np.mean((salt_sense_data_exp-mean_uq_)**2)
    
    salt_sense_data = np.mean(selected_pred_uq_subsampled, axis=1)
    
    figure_output_sens_uq__ = plt.figure('Sensitivity of Positive Ion Density at Z')
    ax_sens_uq__ = figure_output_sens_uq__.axes
    ax_sens_uq__[0].cla()
    ax_sens_uq__[1].cla()
    #ax_sens_uq__[2].cla()   
    
    #ax_sens_uq__[1].set_xlabel('Density (M)', fontsize=9)
    #ax_sens_uq__[1].set_ylabel('Probability Density (PD)', fontsize=9)
    #ax_sens_uq__[2].set_xlabel('Density (M)', fontsize=9)
    #figure_output_sens_uq__.text(0.5, 0.02, 'Density (M)', ha='center', fontsize=8)
    #figure_output_sens_uq__.text(0.005, 0.5, 'Probability Density (PD)', va='center', rotation='vertical', fontsize=8)
    
    #ax_sens_uq__[1].set_title('Salt concentration')
    #count2, bins2, ignored2 = ax_sens_uq__[1].hist(salt_sense_data, bins=50, density=True, alpha=0.8, color = "magenta", ec="black")
    
    #ax_sens_uq__[2].set_title('Ion diameter')
    #count1, bins1, ignored1 = ax_sens_uq__[2].hist(dimeter_sense_data, bins=50, density=True, alpha=0.8, color = "cyan", ec="black") #, ec="black"
    
    sens_salt_uq_mean.value=salt_sense_data_var
    sens_salt_uq_std.value=salt_sense_data_var/var_uq_
    
    #plot a gauusian with mu and sigma
    #x_uq_1 = np.linspace(sens_salt_uq_mean.value - 3*sens_salt_uq_std.value, sens_salt_uq_mean.value + 3*sens_salt_uq_std.value, 100)
    #ax_sens_uq__[1].plot(x_uq_1, stats.norm.pdf(x_uq_1, sens_salt_uq_mean.value, sens_salt_uq_std.value), color = "blue", alpha=0.4, linestyle='-')
    
    sens_diam_uq_mean.value=dimeter_sense_data_var
    sens_diam_uq_std.value=dimeter_sense_data_var/var_uq_
    
    #plot a gauusian with mu and sigma
    #x_uq_2 = np.linspace(sens_diam_uq_mean.value - 3*sens_diam_uq_std.value, sens_diam_uq_mean.value + 3*sens_diam_uq_std.value, 100)
    #ax_sens_uq__[2].plot(x_uq_2, stats.norm.pdf(x_uq_2, sens_diam_uq_mean.value, sens_diam_uq_std.value), color = "blue", alpha=0.4, linestyle='-')

    
    #objects_sen_uq = ['Ion diameter', 'Salt concentration']
    objects_sen_uq = ['C', 'D']
    y_pos_sen_uq = np.arange(len(objects_sen_uq))
    
    if not (sens_salt_uq_std.value ==0.0 and sens_diam_uq_std.value ==0.0):
        performance_sen_uq = [salt_sense_data_var/var_uq_, dimeter_sense_data_var/var_uq_]
        performance_sen_uq_pie = [salt_sense_data_var/var_uq_*100, dimeter_sense_data_var/var_uq_*100]
    else:
        performance_sen_uq=[0, 0]
        performance_sen_uq_pie==[0, 0]
    
    bar_colors = ['C0','magenta']
    ax_sens_uq__[0].bar(y_pos_sen_uq, performance_sen_uq, align='center', alpha=0.8, ec="black", color=bar_colors) #,width=0.4
    ax_sens_uq__[0].set_xticks(y_pos_sen_uq)
    ax_sens_uq__[0].set_xticklabels(objects_sen_uq)
    ax_sens_uq__[0].set_ylabel('Sensitivity')
    #ax_sens_uq__[0].set_title('Sensitivity graph')
    
    explode = (0, 0.05)
    ax_sens_uq__[1].pie(performance_sen_uq_pie, explode=explode, labels=objects_sen_uq, autopct='%1.1f%%', shadow=True, startangle=90, colors=bar_colors)
    ax_sens_uq__[1].axis('equal') 
    
    plt.tight_layout()
    
    
def click_uq_contact_button(b):
    global contact_point
    UQ_localOutput.value += "Moving slider to contact density point.\n"
    imageSlider_uq_output.value = contact_point
    UQ_localOutput.value += "Plotting positive ion distribution of contact density point.\n"


## UQ_UI

In [ ]:
%%capture

build_Plot_UQ = True

tab_layout_uq=Layout(
    #width='100%',
    height='643.5px',
    justify_content='space-between',
)

box_layout_ug_group=Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    padding='5px',
    width='100%'
    
)

box_layout_ug_group_column=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    padding='5px'
)

box_layout_uq=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    width='100%'
    #min_height='20px',
    #min_width='200px'
    #width=left_column,
)


################################################### Salt concentration #################################################
label_style = "style='background-color: #498dbc; font-size:500; padding: 2px'"
salt_uq_label = make_label('Salt concentration (M): ', label_style, '')
salt_uq_label.layout = show_layout
graph_icon = widgets.HTML('<i class="fa fa-area-chart"></i>')
graph_icon.layout = show_layout

salt_uq_select_drop = widgets.Dropdown(
    options=['gaussian', 'uniform','exact'],
    value='gaussian',
    disabled=False,
    description='Distribution:',
    icon='area-chart',
    layout = show_layout
)

salt_uq_1 = widgets.BoundedFloatText(
    value=.6,
    min=.1,
    max=2.0,
    step=0.1,
    description='Mean:',
    style=style,
    layout = show_layout
)

salt_uq_2 = widgets.BoundedFloatText(
    value=.001,
    min=.0001,
    max=.1,
    step=0.0001,
    description='Std.:',
    style=style,
    layout = show_layout
)


def salt_select_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new']== 'gaussian':
            salt_uq_1.layout = show_layout
            salt_uq_2.layout = show_layout
            salt_uq_1.description='Mean:'
            salt_uq_1.max=2.0
            salt_uq_1.min=0.1
            salt_uq_1.step=0.1
            salt_uq_1.value=0.6
            salt_uq_2.description='Std.:'
            salt_uq_2.min=0.0001
            salt_uq_2.max=0.1
            salt_uq_2.step=0.0001
            salt_uq_2.value=0.001
        elif change['new']== 'uniform':
            salt_uq_1.layout =show_layout
            salt_uq_2.layout = show_layout
            salt_uq_1.description='Min:'
            salt_uq_1.max=2.0
            salt_uq_1.min=0.1
            salt_uq_1.step=0.1
            salt_uq_1.value=0.5
            salt_uq_2.description='Max:'
            salt_uq_2.max=2.0
            salt_uq_2.min=0.1
            salt_uq_2.step=0.1
            salt_uq_2.value=0.7
        elif change['new']== 'exact':
            salt_uq_1.layout = show_layout
            salt_uq_2.layout = hide_layout
            salt_uq_1.description='Value:'
            salt_uq_1.max=2.0
            salt_uq_1.min=0.1
            salt_uq_1.step=0.1 
            salt_uq_1.value=0.6
                   
salt_uq_select_drop.observe(salt_select_drop_on_change)


salt_uq_form = [
    Box([graph_icon, salt_uq_select_drop, salt_uq_1, salt_uq_2], layout=box_layout_ug_group)
]
################################################### Diameter #################################################
label_style = "style='background-color: #f230f2; font-size:500; padding: 2px'"
ion_diam_uq_label = make_label('Ion diameter (nm): ', label_style, '')
ion_diam_uq_label.layout = show_layout
ion_diam_graph_icon = widgets.HTML('<i class="fa fa-area-chart"></i>')
ion_diam_graph_icon.layout = show_layout

ion_diam_uq_select_drop = widgets.Dropdown(
    options=['gaussian', 'uniform','exact'],
    value='gaussian',
    disabled=False,
    description='Distribution:',
    icon='area-chart',
    layout = show_layout
)

ion_diam_uq_1 = widgets.BoundedFloatText(
    value=.627,
    min=.2,
    max=.63,
    step=0.001,
    description='Mean:',
    style=style,
    layout = show_layout
)

ion_diam_uq_2 = widgets.BoundedFloatText(
    value=.001,
    min=.0001,
    max=.1,
    step=0.0001,
    description='Std.:',
    style=style,
    layout = show_layout
)


def ion_diam_select_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new']== 'gaussian':
            ion_diam_uq_1.layout = show_layout
            ion_diam_uq_2.layout = show_layout
            ion_diam_uq_1.description='Mean:'
            ion_diam_uq_1.max=.63
            ion_diam_uq_1.min=.2
            ion_diam_uq_1.step=0.001
            ion_diam_uq_1.value=.627
            ion_diam_uq_2.description='Std.:'
            ion_diam_uq_2.min=0.0001
            ion_diam_uq_2.max=0.1
            ion_diam_uq_2.step=0.0001
            ion_diam_uq_2.value=0.001
        elif change['new']== 'uniform':
            ion_diam_uq_1.layout = show_layout
            ion_diam_uq_2.layout = show_layout
            ion_diam_uq_1.description='Min:'
            ion_diam_uq_1.max=.63
            ion_diam_uq_1.min=.2
            ion_diam_uq_1.step=0.001
            ion_diam_uq_1.value=.55
            ion_diam_uq_2.description='Max:'
            ion_diam_uq_2.max=.63
            ion_diam_uq_2.min=.2
            ion_diam_uq_2.step=0.001
            ion_diam_uq_2.value=.65
        elif change['new']== 'exact':
            ion_diam_uq_1.layout = show_layout
            ion_diam_uq_2.layout = hide_layout
            ion_diam_uq_1.description='Value:'
            ion_diam_uq_1.max=.63
            ion_diam_uq_1.min=.2
            ion_diam_uq_1.step=0.001 
            ion_diam_uq_1.value=.627
                   
ion_diam_uq_select_drop.observe(ion_diam_select_drop_on_change)


ion_diam_uq_form = [
    Box([ion_diam_graph_icon, ion_diam_uq_select_drop, ion_diam_uq_1, ion_diam_uq_2], layout=box_layout_ug_group)

]

samples_per_parameter = widgets.IntText(
    value=100,
    description='Samples per input:',
    disabled=False,
    min=100,
    max=1000,
    step=10,
    style=style,
    #layout = show_layout
)

samples_per_parameter2 = widgets.IntText(
    value=100,
    description='input:',
    disabled=False,
    min=100,
    max=1000,
    step=10,
    style=style,
    layout = hide_layout
)

simulate_uq = widgets.Button(
    description='Simulate',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run UQ process',
    icon='area-chart',
    #width=300,
    #height=400,
    #layout=widgets.Layout(width='auto', height=salt_uq.layout.height),
    disabled=False,
    style=style,
    layout = show_layout
)

simulate_uq.on_click(click_simulate_uq)


box_layout_ug_group_run_btn=Layout(
    display='flex',
    flex_flow='row',
    padding='5px',
    width='100%',
    align_items='center',
    #margin='0px 0px 0px 100px'
    
)
#flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset'

uq_input_controls = [
    Box([samples_per_parameter], layout=box_layout_ug_group),
    Box([samples_per_parameter2], layout=box_layout_ug_group),
    Box([simulate_uq], layout=box_layout_ug_group_run_btn)

]


form_items_UQ = [
    Box([salt_uq_label], layout=form_item_layout),
    Box(salt_uq_form, layout=form_item_layout),
    Box([ion_diam_uq_label], layout=form_item_layout),
    Box(ion_diam_uq_form, layout=form_item_layout),
    Box(uq_input_controls, layout=form_item_layout)
]


##################################################### Input Graphs #########################################################

tab_layout_inputs_uq_graph=Layout(
    width='50%',
    height='300px',
    padding='5px',
    justify_content='space-between',
)


figsize_tup_uq_input=(3.86, 2.0)
figsize_tup_uq_input2=(3.86, 2.0)
plt.ioff() #LINE ADDED, Interactive plot blur fix
figure_salt_input_uq = plt.figure(num='Salt concentration', figsize=figsize_tup_uq_input)
out_salt_input_uq = Output(layout=tab_layout_inputs_uq_graph)

figure_diameter_input_uq = plt.figure(num='Ion diameter', figsize=figsize_tup_uq_input2)
out_diameter_input_uq = Output(layout=tab_layout_inputs_uq_graph)
plt.ion() #LINE ADDED, Interactive plot blur fix  

uq_input_graphs = [
    Box([out_salt_input_uq, out_diameter_input_uq], layout=box_layout_ug_group)

]

##################################################### Progress bar and output print #################################################################
progressBar_uq=widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout={'width': '100%'},
    style=style
)

UQ_localOutput = widgets.Textarea(layout={'width': '100%', 'height': '150px'})
UQ_localOutputWidget = widgets.Accordion(children=[UQ_localOutput])
UQ_localOutputWidget.set_title(0, "Output")
UQ_localOutputWidget.selected_index = None
UQ_localOutput.value = ""

box_layout_ug_group_Accordion=Layout(
    display='flex',
    #flex_flow='row',
    align_items='stretch',
    padding='5px',
    width='100%'  
)

form_items_progress_uq = [
    Box([progressBar_uq], layout=form_item_layout),
    widgets.VBox([UQ_localOutputWidget], layout=box_layout_ug_group_Accordion)
]

progressBar_UQ_Element = Box(form_items_progress_uq, layout=box_layout_uq)


##################################################### Outputs and Output Graphs #########################################################

tab_layout_output_uq_graph=Layout(
    width='85%',
    height='300px',
    padding='5px',
    justify_content='space-between',
)

float_item_width_out='200px'
hide_layout_btn_output=Layout(
    visibility='hidden', #visible/hidden
    width=float_item_width_out
    
)
show_layout_btn_output=Layout(
    visibility='visible', #visible/hidden   
    width=float_item_width_out
)

imageSlider_uq_text = widgets.HTML('<p>Select a Z point (nm):</p>')
imageSlider_uq_text.layout = hide_layout

imageSlider_uq_output = widgets.FloatSlider(
    value=0,
    min=0,
    max=0,
    step=0,
    description='',
    continuous_update=False,
    style=style
)

imageSlider_uq_output.observe(onUQ_SliderChange, names='value')

imageSlider_uq_output.layout = hide_layout

uq_contact_button = widgets.Button(
    value=False,
    description='Contact density',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clicking on this button would move the slider to contact density point.',
    icon='',
    style=style,
    layout = show_layout
)

uq_contact_button.on_click(click_uq_contact_button)

uq_contact_button.layout = hide_layout

out_uq_dist_inf = make_label('Output distribution info:', label_style2, '')

out_uq_dist_inf.layout = hide_layout 

output_uq_mean = widgets.BoundedFloatText(
    value=0,
    description='Density Mean:',
    style=style,
    layout = hide_layout_btn_output,
    disabled=True
)

output_uq_var = widgets.BoundedFloatText(
    value=0,
    description='Density Var:',
    style=style,
    layout = hide_layout_btn_output,
    disabled=True
)

output_uq_std = widgets.BoundedFloatText(
    value=0,
    description='Density Std.:',
    style=style,
    layout = hide_layout_btn_output,
    disabled=True
)


figsize_tup_uq_output=(5.30, 2.2)
plt.ioff() #LINE ADDED, Interactive plot blur fix
figure_output_uq = plt.figure(num='Positive Ion Density Distribution at Z', figsize=figsize_tup_uq_output)
out_output_uq = Output(layout=tab_layout_output_uq_graph)    
plt.ion() #LINE ADDED, Interactive plot blur fix  

output_items_uq_left = [
    Box([imageSlider_uq_text], layout=form_item_layout),
    Box([imageSlider_uq_output], layout=form_item_layout),
    Box([uq_contact_button], layout=form_item_layout),
    Box([out_uq_dist_inf], layout=form_item_layout),
    Box([output_uq_mean], layout=form_item_layout),
    Box([output_uq_var], layout=form_item_layout),
    Box([output_uq_std], layout=form_item_layout)
]

output_uq_left = Box(output_items_uq_left, layout=box_layout_ug_group_column)

uq_output_items = [
    Box([output_uq_left, out_output_uq], layout=box_layout_ug_group)
]

##################################################### Sensitivity Analysis #########################################################
tab_layout_output_sens_uq_graph=Layout(
    width='75%',
    height='300px',
    padding='5px',
    justify_content='space-between',
)

float_item_width='250px'
hide_layout_btn_uq=Layout(
    visibility='hidden', #visible/hidden
    width=float_item_width
    
)
show_layout_btn_uq=Layout(
    visibility='visible', #visible/hidden   
    width=float_item_width
)
label_style = "style='background-color: #498dbc; font-size:500; padding: 2px'"
sens_salt_label_inf = make_label('Concentration (C):', label_style, '')

sens_salt_label_inf.layout = hide_layout 

sens_salt_uq_mean = widgets.BoundedFloatText(
    value=0,
    description='Var(*):',
    style=style,
    layout = hide_layout_btn_uq,
    disabled=True
)

sens_salt_uq_std = widgets.BoundedFloatText(
    value=0,
    description='Sensitivity:',
    style=style,
    layout = hide_layout_btn_uq,
    disabled=True
)

label_style = "style='background-color: #f230f2; font-size:500; padding: 2px'"
sens_diam_label_inf = make_label('Ion diameter (D):', label_style, '')

sens_diam_label_inf.layout = hide_layout 

sens_diam_uq_mean = widgets.BoundedFloatText(
    value=0,
    description='Var(*):',
    style=style,
    layout = hide_layout_btn_uq,
    disabled=True
)

sens_diam_uq_std = widgets.BoundedFloatText(
    value=0,
    description='Sensitivity:',
    style=style,
    layout = hide_layout_btn_uq,
    disabled=True
)


figsize_tup_sens_uq_output=(5.40, 2.2)#(5.4, 2.0)
plt.ioff() #LINE ADDED, Interactive plot blur fix
#figure_output_sens_uq, ax_sens_uq = plt.subplots(nrows=1, ncols=3, num='Sensitivity of Positive Ion Density at Z', figsize=figsize_tup_sens_uq_output)
figure_output_sens_uq = plt.figure(num="Sensitivity of Positive Ion Density at Z", figsize=figsize_tup_sens_uq_output)
# cutom subplot
plt.subplot2grid((1,4), (0,0), colspan=2)
plt.subplot2grid((1,4), (0,2), colspan=2)
#plt.subplot2grid((1,5), (0,1), colspan=2)
#plt.subplot2grid((1,5), (0,3), colspan=2)
out_output_sens_uq = Output(layout=tab_layout_output_sens_uq_graph)    
plt.ion() #LINE ADDED, Interactive plot blur fix
#clearing intial graphs
figure_output_sens_uq=plt.figure('Sensitivity of Positive Ion Density at Z')
ax_sens_uqs = figure_output_sens_uq.axes
ax_sens_uqs[0].axis("off")#remove()#cla()
ax_sens_uqs[1].axis("off")

output_items_sens_uq_left = [
    Box([sens_salt_label_inf], layout=form_item_layout),
    Box([sens_salt_uq_mean], layout=form_item_layout),
    Box([sens_salt_uq_std], layout=form_item_layout),
    Box([sens_diam_label_inf], layout=form_item_layout),
    Box([sens_diam_uq_mean], layout=form_item_layout),
    Box([sens_diam_uq_std], layout=form_item_layout)
]

output_sens_uq_left = Box(output_items_sens_uq_left, layout=box_layout_ug_group_column)

sens_uq_output_items = [
    Box([output_sens_uq_left, out_output_sens_uq], layout=box_layout_ug_group)
]


##################################################### Outputs Desnsity plot #########################################################

tab_layout_output_uq_graph_2d=Layout(
    width='100%',
    height='430px',
    padding='5px',
    justify_content='space-between',
)

figsize_tup_uq_output_2d=(7.95, 3.5)
plt.ioff() #LINE ADDED, Interactive plot blur fix
figure_output_uq_2d = plt.figure(num='Distribution of Ion Density for UQ', figsize=figsize_tup_uq_output_2d)
out_output_uq_2d = Output(layout=tab_layout_output_uq_graph_2d)    
plt.ion() #LINE ADDED, Interactive plot blur fix  

uq_output_items_2d = [
    Box([out_output_uq_2d], layout=box_layout_ug_group)
]

##################################################### All elements #########################################################
    
UQ_para = Box(form_items_UQ, layout=box_layout_uq)
UQ_para_graph = Box(uq_input_graphs, layout=box_layout_uq)
UQ_output = Box(uq_output_items, layout=box_layout_uq)
UQ_Sense_output = Box(sens_uq_output_items, layout=box_layout_uq)
UQ_output_2d = Box(uq_output_items_2d, layout=box_layout_uq)

input_controls_text = make_label('Input controls:', label_style2, '')
input_progress_text = make_label('Output information:', label_style2, '')
input_distributions_text = make_label('Input distributions:', label_style2, '')
output_controls_text = make_label('Output controls and distributions:', label_style2, '')
output_sensitivity_text = make_label('Sensitivity analysis:', label_style2, '')
output_controls_text_2d = make_label('All simulations in a 2D histogram plot :', label_style2, '')

UQ_Sec_ = Box([input_controls_text, UQ_para, input_progress_text, progressBar_UQ_Element, input_distributions_text, UQ_para_graph, output_controls_text, UQ_output, output_sensitivity_text, UQ_Sense_output, output_controls_text_2d, UQ_output_2d], layout=box_layout_uq)

tab_Scroll_layout = Layout(overflow='scroll',
                    border='',
                    width='auto',
                    #height='1500px',
                    flex_flow='row',
                    display='flex')

UQ_Sec = Box([UQ_Sec_], layout=tab_Scroll_layout)
#UQ_Sec


# binding ion diameters together
widgets.jslink((ion_diam_pos, 'value'), (ion_diam_uq_1, 'value'))
widgets.jslink((salt, 'value'), (salt_uq_1, 'value'))

In [ ]:
#tabChange detection and plot data incase automatically not drawn
def onTabChange(b):
    global build_Plot, build_Plots
    
    if b['new'] == 6:
        about.layout.height='100px'
        #about_text.layout.padding='0px'
    else:
        about.layout.height='310px'
        
    if os.path.isfile('data/p_density_profile.dat') and os.path.isfile('data/n_density_profile.dat') and not build_Plots:
        if predictionBtn.value and validateParamsMLSurrogate1():
            if not build_Plot:
                plot_data_with_prediction()
        else:
            plot_data()

In [ ]:
def onSliderChange(b):
    
    value = imageSlider.value
    valueText = str(value)
    if value==0:
        valueText = "000"
    
    fileName="image."+valueText+".jpg"
    fileName="outfiles/"+fileName;
    try:
        file = open(fileName, "rb")
        selectedImage = file.read()
        output_image.value=selectedImage
    except:
        sys.stderr.write('Image file is not there yet.')
       
    
imageSlider.observe(onSliderChange, names='value')

In [ ]:
# tab_contents = ['Positive Ion Density', 'Negative Ion Density','Simulation Snapshot', 'Downloads']
# children = [out_desnity_pos, out_desnity_neg, output_image_section, downloadMovieSec]

tab_contents = ['Positive Ion Density', 'Negative Ion Density', 'Net Charge Density', 'Integrated Charge', 'Prediction Graph','Simulation Snapshot','UQ', 'Downloads']
#children = [out_desnity_pos, out_desnity_neg, prediction_graph, output_image_section, UQ_Sec, downloadMovieSec]
children = [out_desnity_pos, out_desnity_neg, out_net_charge_density, out_desnity_integrated_charge, output_prediction_graph_section, output_image_section, UQ_Sec, downloadMovieSec]

tabSpace = widgets.Tab()
tabSpace.children = children
for i in range(len(children)):
    tabSpace.set_title(i, str(tab_contents[i]))
tabSpace.selected_index = 4 #2

tabSpace.observe(onTabChange, names='selected_index')

form_tab_content = [
    Box([tabSpace], layout=form_item_layout_tab)
]

tab = Box(form_tab_content, layout=box_layout_tabs)

In [ ]:
def showPlotUI():
    with out_desnity_pos:
        display(figure_density_pos, layout=tab_layout)
    with out_desnity_neg:
        display(figure_density_neg, layout=tab_layout) 
    with out_net_charge_density:    
        display(figure_net_charge_density, layout=tab_layout)
    with out_desnity_integrated_charge:    
        display(figure_integrated_charge, layout=tab_layout)
#     with prediction_graph:
#         display(figure_density_pre, layout=tab_layout)

def showPlotUIUQ():
    with out_salt_input_uq:
        display(figure_salt_input_uq, layout=tab_layout_inputs_uq_graph)
    with out_diameter_input_uq:
        display(figure_diameter_input_uq, layout=tab_layout_inputs_uq_graph)        
    with out_output_uq:
        display(figure_output_uq, layout=tab_layout_output_uq_graph)
    with out_output_sens_uq:
        display(figure_output_sens_uq, layout=tab_layout_output_sens_uq_graph)        
    with out_output_uq_2d:
        display(figure_output_uq_2d, layout=tab_layout_output_uq_graph_2d)         

In [ ]:
form_item_layout_output = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

form_item_layout_run_btn = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

#Submit(start_func=callExe, done_func=plot_data, cachename='SubmitTest1')
submit_btn=Submit(start_func=callExe, done_func=doneFunc, outcb=stdoutput_handler)
#submit_btn=RunCommand(start_func=callExe, done_func=doneFunc, outcb=stdoutput_handler)
submit_btn.acc.set_title(0, "LAMMPS Output")
submit_btn.w.layout = form_item_layout_run_btn
#submit_btn.acc.layout = form_item_layout_output
localOutput = widgets.Textarea(layout={'width': '100%', 'height': '250px'})
localOutputWidget = widgets.Accordion(children=[localOutput],layout=form_item_layout_output)
localOutputWidget.set_title(0, "Output")
localOutputWidget.selected_index = None
localOutput.value = ""
#vars(submit_btn)

def get_submitbtn_state(b):
    if submit_btn.but.description == 'Cancel':
        salt.disabled = True
        pion.disabled = True
        nion.disabled = True
        clen.disabled = True
        ion_diam_pos.disabled = True
        ion_diam_neg.disabled = True
        charge_on_wall.disabled = True
    else:
        salt.disabled = False
        pion.disabled = False
        nion.disabled = False
        clen.disabled = False
        ion_diam_pos.disabled = False
        ion_diam_neg.disabled = False
        charge_on_wall.disabled = False
    
submit_btn.but.on_click(get_submitbtn_state)


In [ ]:
progressBar=widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout={'width': '100%'},
    style=style
)

form_items_progress = [
    Box([progressBar], layout=form_item_layout)
]

progressBar_Ele = Box(form_items_progress, layout=box_layout_computing)

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

box_layout_column_left=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=left_column,
)
box_layout_column_right=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=right_column
)

box_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height='1080px'
)

box_layout_rows=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height='2150px'#'1770px',
    #overflow_x='scroll',
    #overflow=None
)


gui_left = [
    Box([mainImage], layout=form_item_layout),
    Box([physical_para], layout=form_item_layout),
    Box([computing_para], layout=form_item_layout),
    Box([progressBar_Ele], layout=form_item_layout),
    Box([output_para], layout=form_item_layout)
]

gui_right = [
    Box([about_text], layout=form_item_layout),
    Box([tab], layout=form_item_layout)
]

form_left = Box(gui_left, layout=box_layout_column_left)
form_right = Box(gui_right, layout=box_layout_column_right)

gui_upper = [
    Box([form_left], layout=box_layout_column_left),
    Box([form_right], layout=box_layout_column_right)
]

form = Box(gui_upper, layout=box_layout)

gui = widgets.VBox([form,localOutputWidget,submit_btn.w],layout=box_layout_rows)


In [ ]:
gui